In [ ]:
######################17/11/2022 

####author: weiyi liu
####拆分出的子功能---》剪裁pdf至图片部分

##18/11/2022
########################

###！@input

pdfname="vosTimbres05122022"



###  pdf    name
###！@output
    #SOUSIMGS   相当于做了一个剪裁的动作





###库
import cv2
import numpy as np
from datetime import datetime
import pytz
from math import *
##识别图片模型（下载了很久。。。。。）
import os, shutil
import img2pdf
import glob
import fitz
import re
import PyPDF2
from PIL import Image,ImageFont,ImageDraw
import matplotlib.pyplot as plt
STACK=[]
pdf_header="pdfs/"
#############

def mkdir(path):
    folder = os.path.exists(path)
    if not folder:  # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  # makedirs 创建文件时如果路径不存在会创建这个路径
    else:
        pass


def pdf_image(pdfPath, imgPath, zoom_x, zoom_y, rotation_angle):
    """
    :param pdfPath: pdf文件的路径
    :param imgPath: 图像要保存的文件夹
    :param zoom_x: x方向的缩放系数
    :param zoom_y: y方向的缩放系数
    :param rotation_angle: 旋转角度
    :return: None                                   
    """
    # 打开PDF文件

    pdf = fitz.open(pdfPath)
    pdfReader = PyPDF2.PdfFileReader(pdfPath)
    name = pdf.name
    ###将前面的文件夹路径替换，以及把后缀的地方去掉
    name = name.replace('pdfs/', '').replace('.pdf', '')
    # 逐页读取PDF
    ###有一些小函数在小细节上面做出了更新
    for pg in range(0, pdf.page_count):
        page = pdf.load_page(pg)

        ###########add-stack
        pagehandle = pdfReader.getPage(pg)
        pagestack=re.findall("SD.*",pagehandle.extractText())
        STACK.append(pagestack)
        # 设置缩放和旋转系数
        trans = fitz.Matrix(zoom_x, zoom_y).prerotate(rotation_angle)
        pm = page.get_pixmap(matrix=trans, alpha=False)
        # 开始写图像
        mkdir(imgPath + name)
        ###现在直接用save函数保存图片了、
        pm.save(imgPath + name + '/' + str(pg + 1) + ".jpg")
    pdf.close()
    shutil.move(pdfPath,"./used-pdf/")

pdf_image(pdf_header+pdfname+'.pdf', r"images/", 5, 5, 0)





###########process1 finished
###val=>stack[size 3]

###########now process2 start
##file_ficher
imgfiles="./images/"
head=imgfiles+pdfname+'/'
file_dir=imgfiles+pdfname+'/'



####################################start
##############################################################################
##############################################################################
#这里的话要尝试从两个角度解决遍历排序问题
file_list = []
for items in os.walk(file_dir, topdown=False):
    file_list = items[2]
print(file_list)
##写一个基于re的冒泡排序法将结果进行排序
def bubble_sort(alist):
    # 结算列表的长度
    n = len(alist)
    # 外层循环控制从头走到尾的次数
    for j in range(n - 1):
        # 用一个count记录一共交换的次数，可以排除已经是排好的序列；这样可以加快程序速度
        count = 0
        # 内层循环控制走一次的过程
        for i in range(0, n - 1 - j):
            # 如果前一个元素大于后一个元素，则交换两个元素（升序）
            if int(re.findall("\d+",file_list[i])[0]) > int(re.findall("\d+",file_list[i+1])[0]):
                # 交换元素
                alist[i], alist[i + 1] = alist[i + 1], alist[i]
                # 记录交换的次数
                count += 1
        # count == 0 代表没有交换，序列已经有序
        if 0 == count:
            break
bubble_sort(file_list)
print(file_list)















#排序完毕，开始分割
filelistcount=0
for file in file_list:
    pic_path =head+file # 分割的图片的位置
    pic_target = './sousimgs/' # 分割后的图片保存的文件夹
    if not os.path.exists(pic_target):  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(pic_target)
    picture = cv2.imread(pic_path)
        ###测试过后得出的长度和宽度的参数
    cut_width = 520
    cut_length = 992
    # 读取要分割的图片，以及其尺寸等数据
    picture = cv2.imread(pic_path)
    (width, length, depth) = picture.shape
    # 预处理生成0矩阵
    pic = np.zeros((cut_width, cut_length, depth))
    # 计算可以划分的横纵的个数
    #num_width = int(width / cut_width)
    #num_length = int(length / cut_length)
    # for循环迭代生成
    timbrecount=0
    for i in range(0, 8):
        for j in range(0, 3):
            pic = picture[i*cut_width : (i+1)*cut_width, j*cut_length : (j+1)*cut_length, :]      
            ####命名这里暂时先用前面文件名代替
            ####由于分号不能出现因此暂时用-号替换
            #####直接不要前三位了
            result=STACK[filelistcount][timbrecount][5:]
            result_path = pic_target + '{}.jpg'.format(result)
            cv2.imwrite(result_path, pic)
            timbrecount+=1
    filelistcount+=1
###用它生成过图片了，移动到使用完的文件夹中
shutil.move(file_dir ,"./used-pdfpage/")






In [ ]:
#######################logs

In [ ]:



###########process1 finished
###val=>stack[size 3]

###########now process2 start
##file_ficher
imgfiles="./images/"
head=imgfiles+pdfname+'/'
file_dir=imgfiles+pdfname+'/'



####################################start
##############################################################################
##############################################################################
#这里的话要尝试从两个角度解决遍历排序问题
file_list = []
for items in os.walk(file_dir, topdown=False):
    file_list = items[2]
print(file_list)
##写一个基于re的冒泡排序法将结果进行排序
def bubble_sort(alist):
    # 结算列表的长度
    n = len(alist)
    # 外层循环控制从头走到尾的次数
    for j in range(n - 1):
        # 用一个count记录一共交换的次数，可以排除已经是排好的序列；这样可以加快程序速度
        count = 0
        # 内层循环控制走一次的过程
        for i in range(0, n - 1 - j):
            # 如果前一个元素大于后一个元素，则交换两个元素（升序）
            if int(re.findall("\d+",file_list[i])[0]) > int(re.findall("\d+",file_list[i+1])[0]):
                # 交换元素
                alist[i], alist[i + 1] = alist[i + 1], alist[i]
                # 记录交换的次数
                count += 1
        # count == 0 代表没有交换，序列已经有序
        if 0 == count:
            break
bubble_sort(file_list)
print(file_list)















#排序完毕，开始分割
filelistcount=0
for file in file_list:
    pic_path =head+file # 分割的图片的位置
    pic_target = './sousimgs/' # 分割后的图片保存的文件夹
    if not os.path.exists(pic_target):  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(pic_target)
    picture = cv2.imread(pic_path)
        ###测试过后得出的长度和宽度的参数
    cut_width = 520
    cut_length = 992
    # 读取要分割的图片，以及其尺寸等数据
    picture = cv2.imread(pic_path)
    (width, length, depth) = picture.shape
    # 预处理生成0矩阵
    pic = np.zeros((cut_width, cut_length, depth))
    # 计算可以划分的横纵的个数
    #num_width = int(width / cut_width)
    #num_length = int(length / cut_length)
    # for循环迭代生成
    timbrecount=0
    for i in range(0, 8):
        for j in range(0, 3):
            pic = picture[i*cut_width : (i+1)*cut_width, j*cut_length : (j+1)*cut_length, :]      
            ####命名这里暂时先用前面文件名代替
            ####由于分号不能出现因此暂时用-号替换
            #####直接不要前三位了
            result=STACK[filelistcount][timbrecount][5:]
            result_path = pic_target + '{}.jpg'.format(result)
            cv2.imwrite(result_path, pic)
            timbrecount+=1
    filelistcount+=1
###用它生成过图片了，移动到使用完的文件夹中
shutil.move(file_dir ,"./used-pdfpage/")




In [4]:


###库
import cv2
import numpy as np
from datetime import datetime
import pytz
from math import *
##识别图片模型（下载了很久。。。。。）
import os, shutil
import img2pdf
import glob
import fitz
import re
import PyPDF2
from PIL import Image,ImageFont,ImageDraw
import matplotlib.pyplot as plt
STACK=[]
pdf_header="./used-pdf/"
pdfname="vosTimbres22112022"
pdfpath=pdf_header+pdfname+'.pdf'
pdf = fitz.open(pdfpath)

In [7]:
f=open(pdfpath,"rb")
pdfReader = PyPDF2.PdfFileReader(f)
page = pdf.load_page(0)
pagehandle = pdfReader.getPage(0)
pagestack=re.findall("SD.*",pagehandle.extractText())


In [8]:
pagestack

['SD : 87500009874884R',
 'SD : 87500009874885P',
 'SD : 87500009874886N',
 'SD : 87500009874887L',
 'SD : 87500009874888J',
 'SD : 87500009874889H',
 'SD : 87500009874890V',
 'SD : 87500009874891T',
 'SD : 87500009874892R',
 'SD : 87500009874893P',
 'SD : 87500009909676S',
 'SD : 87500009909677Q',
 'SD : 87500009909678O',
 'SD : 87500009909679M',
 'SD : 875000099096800',
 'SD : 87500009909681Y',
 'SD : 87500009909682W',
 'SD : 87500009909683U',
 'SD : 87500009909684S',
 'SD : 87500009909685Q',
 'SD : 875000099096463',
 'SD : 875000099096471',
 'SD : 875000099096480',
 'SD : 87500009909649Y']

In [12]:
pdfReader.getPage(0).extractText()

'LA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874884R\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874885P\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874886N\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874887L\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874888J\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874889H\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874890V\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874891T\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874892R\nLA POSTETimbre imprimé sur laposte.fr\nFRANCELettre verte\nMax 100g\nFRANCE\nSD : 87500009874893P\nLA POSTET

In [5]:
pagestack
pdf.close()